# Generate representative data of vital kits
For senior kits and home essential kits

In [1]:
# get autocomplete working
%config Completer.use_jedi = False

from abc import ABC, abstractmethod
from enum import Enum

import arrow
import numpy as np

In [7]:
type(arrow.utcnow())

arrow.arrow.Arrow

In [68]:
class VitalKitUser(ABC):
    """
    Abstract base class for all users of a kit.
    
    Presents the `use_kit` method, which randomly selects a kit to use and "uses" it
    """
    @abstractmethod
    def use_kit(self):
        """
        This will select a kit, and use it at a random time after the last use of a kit.
        When the kit is used it returns a set of equal length (possibly singular) time
        series representing the observations from the kit. Time series are of the format
        Tuple[arrow.Arrow, float]
        """
        pass

class Temperature(Enum):
    """
    Enum for representing if a temperature is too high, normal, or to low
    """
    high = 0
    normal = 1
    low = 2

TEMPERATURE_RANGES =\
    [(37.2, 42),     # high temperature range
     (34.7, 37.2),   # normal temperature range
     (32, 34.7)]     # low temperature range
"""
List of the ranges of temperature considered high, normal and low respectively
"""

TEMPERATURE_PROBS = [0.05, 0.9, 0.05]
"""
Default probabilities of temperature being high, normal and low respectively
"""

STANDARD_PULSE_OXIMETER_TIME = 5
"""
The default number of secodns that a user users the pulse oximeter for
"""

STANDARD_HEART_RANGE = (50, 90)
"""
The default heart rate for a user
"""

STANDARD_SPO2_RANGE = (60, 100)
"""
Defautl range of SpO2 for a user
"""

STANDARD_SPO2_HEALTH_FACTOR = 10
"""
The default helath weighting for SpO2, the highest this value is the more likely
that SpO2 will be higher
"""
    
def _use_thermometer(event_probs=None):
    """
    Using a thermometer is modelled as a hierarchical distribution. Firt we decide if
    the temperature is high, normal or low, and depending on that decision we decide
    a respective temperature
    :param event_probs: List of length 3 that details the probability of temperature
    being [high, normal, low], if not given assumed probabilities are equal
    """
    if not event_probs:
        event_probs = TEMPERATURE_PROBS
    if not len(event_probs) == 3:
        raise ValueError("event_prpbs must be a list of probabilities for [high, medium, low]")
    # make sure event probabilities sum to one
    event_probs /= np.sum(event_probs)
    
    # sample categorical distribution for type of temperature to get the temperature range
    temperature_type = np.random.choice(Temperature, p=event_probs)
    temperature_range = TEMPERATURE_RANGES[temperature_type.value]
    
    # sample a temperate for the type of temperature
    return np.random.uniform(*temperature_range)

def _use_pulse_oximeter(*,
                        average_use_time=None,
                        base_heart_rate_range=None,
                        base_spo2_range=None,
                        spo2_health_factor=None):
    """
    Using a thermometer is modelled as a hierarchical distribution. First we decide how
    long a user uses the device for, then we decide what the base heart rate and SpO2 readings
    are going to be, then we generate noisy readings of these base values
    :param average_use_time: the average number of seconds that a user uses the pulse oximeter
    :param base_heart_range: tuple of highest and lowest heart rate for user
    :param base_spo2_range: tuple of highest and lowest SpO2 for user
    :param spo2_health_factor: higher this number is the more likely they are to have a health SpO2 reading
    :returns: tuple of list of heart rate and SpO2 readings
    """
    if not average_use_time:
        average_use_time = STANDARD_PULSE_OXIMETER_TIME
    if not base_heart_rate:
        base_heart_rate_range = STANDARD_HEART_RANGE
    if not base_spo2:
        base_spo2_range = STANDARD_SPO2_RANGE
    if not spo2_health_factor:
        spo2_health_factor = STANDARD_SPO2_HEALTH_FACTOR
    
    # get number of seconds device is used for
    num_secs = np.random.geometric(1/average_use_time)
    
    # get the base heart rate
    base_heart_rate = np.random.uniform(*base_heart_rate_range)
    
    # get base specific oxygen level
    spo2_low, spo2_high = base_heart_rate_range
    spo2_weight = np.random.uniform() ** spo2_health_factor
    base_spo2 = (spo2_weight * spo2_hight) + (spo2_high - spo2_low)